In [1]:
# import des librairies dont nous aurons besoin
import pandas as pd
import numpy as np
import re

# chargement et affichage des données
data = pd.read_csv('personnes.csv')
display(data)

,prenom,email,date_naissance,pays,taille
0,Leila,leila@example.com,23/01/1990,France,1.49m
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m
2,Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,153cm
3,Marc,"marco23@example.com, mc23@supermail.eu",10/02/1978,France,1.65m
4,Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.34m
5,Hanna,hanna2019@supermail.eu,01/01/1970,24,3.45m
6,samuël,samuel_329@example.com,NaN,Bénin,1.45m


# 1. Détecter les erreurs

Détecter les valeurs manquantes :

In [2]:
print(data.isnull().sum())

prenom            0
email             0
date_naissance    1
pays              1
taille            0
dtype: int64


Détecter les doublons :

In [3]:
data.loc[data['email'].duplicated(keep=False),:]

,prenom,email,date_naissance,pays,taille
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m
6,samuël,samuel_329@example.com,NaN,Bénin,1.45m


# 2. Traiter les erreurs

## Traiter la variable Pays

Pour les pays nous avions un problème majeur qui était la valeur 24 , à la 5ème ligne. La première approche pourrait être de considérer que l'ensemble de mes pays doivent être des chaines de caractères et donc toutes les valeurs numériques doivent être remplacées par des valeurs manquantes. Mais pour faire la chose encore plus "proprement", on pourrait définir une liste de pays valides, et remplacer toutes les valeurs qui ne concordent pas avec la liste par des valeurs manquantes :

In [4]:
VALID_COUNTRIES = ['France', 'Côte d\'ivoire', 'Madagascar', 'Bénin', 'Allemagne'
                  , 'USA']
mask = ~data['pays'].isin(VALID_COUNTRIES)
data.loc[mask, 'pays'] = np.NaN

display(data)

,prenom,email,date_naissance,pays,taille
0,Leila,leila@example.com,23/01/1990,France,1.49m
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m
2,Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,153cm
3,Marc,"marco23@example.com, mc23@supermail.eu",10/02/1978,France,1.65m
4,Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.34m
5,Hanna,hanna2019@supermail.eu,01/01/1970,NaN,3.45m
6,samuël,samuel_329@example.com,NaN,Bénin,1.45m


## Traiter les emails

Au tour des emails maintenant ! Le problème avec cette colonne, c'est qu'il y a parfois 2 adresses email par ligne. Nous ne souhaitons prendre que la première. Nous pouvons là aussi utiliser les outils fournies par Pandas pour faire cela :

In [5]:
data['email'] = data['email'].str.split(',', n=1, expand=True)[0]

display(data)

,prenom,email,date_naissance,pays,taille
0,Leila,leila@example.com,23/01/1990,France,1.49m
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m
2,Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,153cm
3,Marc,marco23@example.com,10/02/1978,France,1.65m
4,Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.34m
5,Hanna,hanna2019@supermail.eu,01/01/1970,NaN,3.45m
6,samuël,samuel_329@example.com,NaN,Bénin,1.45m


## Traiter les tailles

Nous aurons besoin d'effectuer deux opérations :

1. convertir les tailles en nombre décimal. Nous en profiterons pour remplacer l'ensemble des tailles non conformes par des valeurs manquantes
2. remplacer les valeurs manquantes par la moyenne de notre échantillon

Pour la première étape, comme l'ensemble de nos valeurs sont censé avoir pour format X.XXm (avec X correspondant à un chiffre), on peut par exemple supprimer le dernier caractère de la colonne et convertir l'ensemble de la colonne en numérique. Ce faisant, les valeurs non valides seront automatiquement remplacées par des valeurs manquantes, via l'argument  errors  que nous spécifions :

In [6]:
data['taille'] = data['taille'].str[:-1]
data['taille'] = pd.to_numeric(data['taille'], errors='coerce')

Ensuite, il faut à présent remplacer les valeurs manquantes par la moyenne de la variable. Cette simple ligne permettra de faire cela :

In [7]:
data.loc[data['taille'].isnull(), 'taille'] = data['taille'].mean()

display(data)

,prenom,email,date_naissance,pays,taille
0,Leila,leila@example.com,23/01/1990,France,1.490000
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.670000
2,Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,1.841667
3,Marc,marco23@example.com,10/02/1978,France,1.650000
4,Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.340000
5,Hanna,hanna2019@supermail.eu,01/01/1970,NaN,3.450000
6,samuël,samuel_329@example.com,NaN,Bénin,1.450000


## Traiter les dates

Dernier sujet à traiter et pas des moindres : les dates ! Ici, nous allons simplement spécifier un certain format de date et transformer notre variable dans un type adéquat :

In [8]:
data['date_naissance'] = pd.to_datetime(data['date_naissance'], format='%d/%m/%Y', errors='coerce')

display(data)

,prenom,email,date_naissance,pays,taille
0,Leila,leila@example.com,1990-01-23,France,1.490000
1,Samuel,samuel_329@example.com,2001-09-20,NaN,1.670000
2,Radia,choupipoune@supermail.eu,NaT,Côte d'ivoire,1.841667
3,Marc,marco23@example.com,1978-02-10,France,1.650000
4,Heri,helloworld@supermail.eu,2008-03-05,Madagascar,1.340000
5,Hanna,hanna2019@supermail.eu,1970-01-01,NaN,3.450000
6,samuël,samuel_329@example.com,NaT,Bénin,1.450000
